In [24]:
import pandas as pd
import time
import datetime

data = pd.read_excel('girdi_test.xlsx')
data['Tipi'] = ""
data['Birlestirilen'] = ""
data["Ilk Sira"] = data.index + 0

t1 = 11 #Başlangıç Zamanı'nın Satırı
t2 = 12 #Bitiş Zamanı'nın Satırı
time_format = "%d.%m.%Y %H:%M:%S" #Değiştirmeye gerek yok

#datetime to timestamp
asama = 0
for i in range(len(data)):    
    if int((i+1)*100/len(data)) > asama:
        asama = int((i+1)*100/len(data))
        print("Timestamp'e Dönüştürülüyor... %", asama, " Tamamlandı.", sep="")
    
    string = str(data.iloc[i,t1])   
    
    try:
        strip = time.strptime(string, time_format)
    except:
        try:
            time_format = "%d-%m-%Y %H:%M:%S"
            strip = time.strptime(string, time_format)
        except:
            try:
                time_format = "%Y.%m.%d %H:%M:%S"
                strip = time.strptime(string, time_format)
            except:
                try:
                    time_format = "%Y-%m-%d %H:%M:%S"
                    strip = time.strptime(string, time_format)
                except:
                    time_format = "%d.%m.%Y %H:%M:%S"
                    strip = time.strptime(string, time_format)
                
    seconds = time.mktime(strip)
    data.iloc[i,t1] = seconds        
    
    string = str(data.iloc[i,t2])
    
    try:
        strip = time.strptime(string, time_format)
    except:
        try:
            time_format = "%d-%m-%Y %H:%M:%S"
            strip = time.strptime(string, time_format)
        except:
            try:
                time_format = "%Y.%m.%d %H:%M:%S"
                strip = time.strptime(string, time_format)
            except:
                try:
                    time_format = "%Y-%m-%d %H:%M:%S"
                    strip = time.strptime(string, time_format)
                except:
                    time_format = "%d.%m.%Y %H:%M:%S"
                    strip = time.strptime(string, time_format)
    
    seconds = time.mktime(strip)
    data.iloc[i,t2] = seconds

data = data.sort_values(by=[data.iloc[:,t1].name])

count_initial = 0
bool_initial = True
asama = 0
while bool_initial:
    if int((count_initial + 1)*100/len(data)) > asama:
        asama = int((count_initial + 1)*100/len(data))
        print("Girişimler Aranıyor... ","%", asama, " Tamamlandı.", sep="")
    count_in = 1
    bool_in = True
    group = []
    recursive = count_initial
    while bool_in: #Sıradaki verinin bitişinden itibaren, 2 saat içerisinde başlamış ve aynı bölgedeki verileri alır.
        if count_in + recursive < len(data):
            if data.iloc[recursive,t2] >= data.iloc[recursive+count_in,t2]:
                if data.iloc[count_initial,5] == data.iloc[recursive + count_in,5]:
                    group.append(recursive + count_in)
                count_in += 1
            elif data.iloc[recursive,t2] >= data.iloc[recursive+count_in,t1] - 7200:
                if data.iloc[count_initial,5] == data.iloc[recursive + count_in,5]:
                    group.append(recursive + count_in)
                    recursive += count_in
                    count_in = 1                    
                else:
                    count_in += 1
                
            else:
                bool_in = False
        else:
            bool_in = False
    
    for i in range (len(group)):        
        count_handle = group[i]
        if data.iloc[count_initial,t2] >= data.iloc[count_handle,t2]: # Tam iç içe  
                data.iloc[count_initial,-2] =  data.iloc[count_initial,-2] + "," + str(data.iloc[count_handle,0])
                data = data.drop([data.index[count_handle]])
                data.iloc[count_initial,-3] = "Tam İç İçe"
                group = [x - 1 for x in group]
        elif data.iloc[count_initial,t2] >= data.iloc[count_handle,t1]: # İç İçe                
                data.iloc[count_initial,t2] = data.iloc[count_handle,t2]
                data.iloc[count_initial,-2] =  data.iloc[count_initial,-2] + "," + str(data.iloc[count_handle,0])
                data = data.drop([data.index[count_handle]])
                data.iloc[count_initial,-3] = "İç İçe"
                group = [x - 1 for x in group]    
        elif data.iloc[count_initial,t2] >= data.iloc[count_handle,t1] - 7200: #Parçalı
                data.iloc[count_initial,t2] = data.iloc[count_handle,t2]
                data.iloc[count_initial,-2] =  data.iloc[count_initial,-2] + "," + str(data.iloc[count_handle,0])
                data = data.drop([data.index[count_handle]])
                data.iloc[count_initial,-3] = "Parçalı"
                group = [x - 1 for x in group]
    count_initial += 1
    
    if count_initial >= len(data)-1:
        bool_initial = False

asama = 0
data = data[data.Tipi != ""]

for i in range(len(data)):
    if int((i+1)*100/len(data)) > asama:
        asama = int((i+1)*100/len(data))
        print("Zaman Formatı'na Dönüştürülüyor... %", asama, " Tamamlandı.", sep="")
    stamp = data.iloc[i,t1]
    value = datetime.datetime.fromtimestamp(stamp)
    data.iloc[i,t1] = value.strftime(time_format)
    
    stamp = data.iloc[i,t2]
    value = datetime.datetime.fromtimestamp(stamp)
    data.iloc[i,t2] = value.strftime(time_format)

data = data.sort_values(by=["Ilk Sira"])
data.pop("Ilk Sira")
data.to_excel("cikti.xlsx")

Timestamp'e Dönüştürülüyor... %3 Tamamlandı.
Timestamp'e Dönüştürülüyor... %6 Tamamlandı.
Timestamp'e Dönüştürülüyor... %9 Tamamlandı.
Timestamp'e Dönüştürülüyor... %12 Tamamlandı.
Timestamp'e Dönüştürülüyor... %16 Tamamlandı.
Timestamp'e Dönüştürülüyor... %19 Tamamlandı.
Timestamp'e Dönüştürülüyor... %22 Tamamlandı.
Timestamp'e Dönüştürülüyor... %25 Tamamlandı.
Timestamp'e Dönüştürülüyor... %29 Tamamlandı.
Timestamp'e Dönüştürülüyor... %32 Tamamlandı.
Timestamp'e Dönüştürülüyor... %35 Tamamlandı.
Timestamp'e Dönüştürülüyor... %38 Tamamlandı.
Timestamp'e Dönüştürülüyor... %41 Tamamlandı.
Timestamp'e Dönüştürülüyor... %45 Tamamlandı.
Timestamp'e Dönüştürülüyor... %48 Tamamlandı.
Timestamp'e Dönüştürülüyor... %51 Tamamlandı.
Timestamp'e Dönüştürülüyor... %54 Tamamlandı.
Timestamp'e Dönüştürülüyor... %58 Tamamlandı.
Timestamp'e Dönüştürülüyor... %61 Tamamlandı.
Timestamp'e Dönüştürülüyor... %64 Tamamlandı.
Timestamp'e Dönüştürülüyor... %67 Tamamlandı.
Timestamp'e Dönüştürülüyor... %70 Tam